In [1]:
%matplotlib inline

from importlib.machinery import SourceFileLoader
import algosdk
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import os
import re
import json
import joblib

# load custome module from path.
covid19_WebScrapes = SourceFileLoader("covid19_WebScrapes", "./scripts/covid19_WebScrapes.py").load_module()
merge_prep_data = SourceFileLoader("merge_data", "./scripts/merge_prep_data.py").load_module()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 78)

# Read in Cases Data on County Level

In [2]:
covid19_county_level = covid19_WebScrapes.TestingData_Scraper()

Testing_DF = covid19_county_level.Get_Final_DF(Impute = True)

Imputing values where errors in cumulative stats.


In [3]:
Testing_DF.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Positive,Deaths
453274,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",6/8/20,51,0
453275,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",6/8/20,26,0
453276,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",6/8/20,637,4
453277,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",6/8/20,22,0
453278,84070020,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,"Weber-Morgan, Utah, US",6/8/20,397,8


# Read in Algorand Blockchain Survey Data

In [4]:
API_KEY=str(np.loadtxt('local_var.txt',dtype=str))[8:]

alg_tx = covid19_WebScrapes.Algorand_Scrape(API_KEY)

algod last round: 7105552
algod time since last round: 2636393509
algod catchup: 0
algod latest version: https://github.com/algorandfoundation/specs/tree/e5f565421d720c6f75cdd186f7098495caf9101f
####################
{'hash': 'ZA2L5ZLBJJSE3TVA7YDQQUJ3WQN67LOZ5TNCBOTLMFGFEWVPPOMQ', 'previousBlockHash': 'IICUM53P34PVMZRNU6RFCAKHGJZLBATYZL4FXSS4UX3OLH4QWJ4A', 'seed': 'IHZU5U5FQFJEJFDAVXS5GCYCFPYEV3B7ODBBCRF5M7GCCYB7KPWQ', 'proposer': '4NRNE5RIGC2UGOMGMDR6L5YMQUV3Q76TPOR7TDU3WEMJLMC6BSBEKPJ2SY', 'round': 7105552, 'period': 0, 'txnRoot': 'KHZD7BBZ5VUQFRXM7PISCJ5QZU2SKNBZAZWNQCGHL4M72IIJKCKQ', 'reward': 125234, 'rate': 26000000, 'frac': 2622520597, 'txns': {'transactions': [{'type': 'pay', 'tx': 'GKRTDY4PTI3MVJZVOB4O27CWQTXWLH6CIH3DOGTH5XT3ZLNJP7UA', 'from': 'PRICEP3G2F5L6ZG5WTJIAKEQW4OJJ3FM4XVFQDZI7M2VBTFVUHTTR2AU2U', 'fee': 1000, 'first-round': 7105550, 'last-round': 7106050, 'noteb64': 'eyJwcmljZV9hbGdvX3VzZCI6MC4yMzYxLCJsYXN0X3RyYWRlX2F0IjoiMjAyMC0wNi0wNlQxNTo0NTowNC45NjUyODVaIiwidGltZXN0

AttributeError: 'list' object has no attribute 'concat'

In [ ]:
Survey_DF = alg_tx.Convert_to_DF()

In [ ]:
Survey_DF_trim = Survey_DF[['gc','gr','gzp','ga','gs','sz','tz','tt','tr','mz','qz','q1','q2','q3','q4','ql']]
Survey_DF_trim.columns = ['Country','Region','Zip','AgeGroup','Gender','Symptomatic',
                          'Tested','Tested_Attempt','Test_Result','Received_Care','Quarantined',
                          'Q_Symptoms','Q_Voluntary','Q_Personal','Q_General','Left_Quarantine']
Survey_DF_trim.tail()

# Read in Wikipedia Area Data

In [4]:
wiki_scraper = covid19_WebScrapes.Wiki_Scrape()

county_areas = wiki_scraper.Scrape_Counties()

Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,Georgia_(U.S._state),Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New_Hampshire,New_Jersey,New_Mexico,New_York,North_Carolina,North_Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode_Island,South_Carolina,South_Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West_Virginia,Wisconsin,Wyoming,

In [5]:
County_Areas = pd.DataFrame(county_areas,columns=['State','County_FIPS','County','Area (sqmi)'])

In [6]:
County_Areas['Area (sqmi)'] = County_Areas['Area (sqmi)'] .apply(lambda x : str(x).replace(',',''))
County_Areas['Area (sqmi)'] = County_Areas['Area (sqmi)'].astype(float)

In [7]:
County_Areas.head()

,State,County_FIPS,County,Area (sqmi)
0,Alabama,001,Autauga County,594.44
1,Alabama,003,Baldwin County,1589.78
2,Alabama,005,Barbour County,884.88
3,Alabama,007,Bibb County,622.58
4,Alabama,009,Blount County,644.78


# Read in Alphabet Location Data

In [8]:
google = covid19_WebScrapes.Alphabet_Scrape_V2()

google_df = google.get_Data(country='United States',country_only=False,state_only=False) #pulls county info only

google_df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
196822,US,United States,Alabama,Autauga County,02/15/20,5.0,7.0,NaN,NaN,-4.0,NaN
196823,US,United States,Alabama,Autauga County,02/16/20,0.0,1.0,-23.0,NaN,-4.0,NaN
196824,US,United States,Alabama,Autauga County,02/17/20,8.0,0.0,NaN,NaN,-27.0,5.0
196825,US,United States,Alabama,Autauga County,02/18/20,-2.0,0.0,NaN,NaN,2.0,0.0
196826,US,United States,Alabama,Autauga County,02/19/20,-2.0,0.0,NaN,NaN,2.0,0.0


# Clean Dataframes, Add FIPS ID's

In [9]:
data_cleaner = covid19_WebScrapes.Clean_Data('manually_pulled/FIPS_Codes_USDA.csv',
                                             'manually_pulled/new_state_mapping.txt')

area_data_cleaned = data_cleaner.Clean_Area_Data(County_Areas)
test_data_cleaned = data_cleaner.Clean_Cases_Data(Testing_DF)
google_data_cleaned = data_cleaner.Clean_Loc_Data(google_df)

# Write out dataframes

In [10]:
folder_name = datetime.datetime.strftime(datetime.datetime.today(),'%d%b%y')
os.mkdir('Processed_Data/'+folder_name)

test_data_cleaned.to_csv('Processed_Data/'+folder_name+'/CountyLevel_Cases_Cleaned.csv',index=False)
#Survey_DF_trim.to_csv('Processed_Data/'+folder_name+'/Survey_Data.csv',index=False)
area_data_cleaned.to_csv('Processed_Data/'+folder_name+'/CountyLevel_Areas_Cleaned.csv',index=False)
google_data_cleaned.to_csv('Processed_Data/'+folder_name+'/CountyLevel_Google_LocData_Cleaned.csv',index=False)

# Now Merge Data and Engineer some features

In [11]:
data_merger = merge_prep_data.Merge_Data()

merged_census_data = data_merger.Merge_Census_Data()
merged_scraped_data = data_merger.Merge_Scraped_Data()

final_merge = data_merger.MERGE_ALL(merged_scraped_data,merged_census_data)

In [12]:
cols_remove = ['State_x','County_FIPS','County','Name','UID','iso2','iso3','code3','Province_State',
               'Country_Region','Combined_Key','country_region_code','country_region','sub_region_1',
               'sub_region_2','date','unique_id','State_y','STATE_FIPS','Unnamed: 0','county','state_FIPS',
              'county_fips']

final_merge.drop(cols_remove,axis=1,inplace=True)

In [13]:
engineer_feats = merge_prep_data.Engineer_Feats(datatype_write_loc='Merged_Data/data_types.json')

DF_out = engineer_feats.Apply_Logic(final_merge)

Adding Proximity Logic...
Normalizing Stats , lagging features...
Interpolating Data...
retail_and_recreation_percent_change_from_baseline


./scripts/merge_prep_data.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF[col] = DF.groupby('state')[col].apply(lambda group: group.interpolate(method=stat_fill_method,axis=0).ffill().bfill())


grocery_and_pharmacy_percent_change_from_baseline
parks_percent_change_from_baseline
transit_stations_percent_change_from_baseline
workplaces_percent_change_from_baseline
residential_percent_change_from_baseline


/Users/petersimone/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
./scripts/merge_prep_data.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF[col] = DF.groupby('state')[col].apply(lambda group: group.interpolate(method=stat_fill_method,axis=0,).ffill().bfill())
./scripts/merge_prep_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

Couldn't convert Admin2 to float
Couldn't convert Date to float
Couldn't convert state to float
Couldn't convert Closest_Big_City to float


In [14]:
cols_keep = ['FIPS','state','Admin2','Lat','Long_','Date',
             'Positive','Deaths','Positive_Cases_PopNormed','Deaths_PopNormed',
             'New_Positive_Cases_PopNormed_Lagged',
             'New_Positive_Cases_PopNormed',
             'Area (sqmi)','population','Proximity','Population_Density','Percent_in_Workforce',
             'Households_per_SqMile',
             'retail_and_recreation_percent_change_from_baseline',
             'grocery_and_pharmacy_percent_change_from_baseline',
             'parks_percent_change_from_baseline',
             'transit_stations_percent_change_from_baseline',
             'workplaces_percent_change_from_baseline',
             'residential_percent_change_from_baseline',
             'Mass gathering restrictions', 'Initial business closure',
             'Educational facilities closed', 'Non-essential services closed',
             'Stay at home order', 'Travel severely limited',
             '%_family_households', '%_single_male_households',
             '%_single_female_households', '%_living_alone',
             'total_household_income', 'household_income_less_than_25_years',
             'household_income_25_to_45_years', 'household_income_45_to_65_years',
             'household_income_65_and_older',
             '%_households_with_earnings_last12',
             '%_households_with_no_earnings_last12', '%_in_poverty',
             '%_in_poverty_18_to_59', '%_in_poverty_60_to_74',
             '%_in_poverty_75_to_85',
             '%_white', '%_black',
             '$_other_race', 
             '%_male', '%_female', 
             '%_male_pop_greater_than_60','%_female_pop_greater_than_60',
             '%_workers_less_than_15_to_work', '%_workers_15_to_45_to_work',
             '%_workers_greater_than_45_to_work',
             '%_drive_alone_to_work', '%_carpool_to_work',
             '%_public_transit_to_work', '%_bus_trolley_to_work', '%_walked_to_work',
             '%_cab_other_means_of_transportation_to_work','jail_incarceration_rate_per_100k']

In [15]:
DF_write = DF_out[cols_keep]

In [16]:
engineer_feats.write_out_json(DF_write)

In [17]:
folder_name = datetime.datetime.strftime(datetime.datetime.today(),'%d%b%y')
if not os.path.exists('Merged_Data/'+folder_name) :
    os.mkdir('Merged_Data/'+folder_name)
    
memory_use = DF_write.memory_usage(deep=True,index=False).sum()    

if memory_use > 1e8 :
    print('splitting dataframes for Github push purposes')
    break_val = int((DF_write.shape[0] * 45000000)/memory_use)
    for i in range(int(np.ceil(DF_write.shape[0]/break_val))) :
        name_write = 'Merged_Data/'+folder_name+'/Final_Merged_Pt{}.csv'.format(i+1)
        chunk_write = DF_write.iloc[break_val*i : break_val*(i+1)]
        chunk_write.to_csv(name_write,index=False)
else :
    DF_write.to_csv('Merged_Data/'+folder_name+'/Final_Merged.csv',index=False) #write out csv file

splitting dataframes for Github push purposes
